In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [29]:
egfr_out = pd.read_csv('../data/crc_egfr_out.csv', index_col=0)
vegf_out = pd.read_csv('../data/crc_vegf_out.csv', index_col=0)
ib_out = pd.read_csv('../data/crc_ib_out.csv', index_col=0)
folfox_out = pd.read_csv('../data/crc_folfox_out.csv', index_col=0)
egfr_out.rename(columns={'os_g_status': 'OS', 'pfs_i_and_m_g_status': 'PFS'}, inplace=True)
vegf_out.rename(columns={'os_g_status': 'OS', 'pfs_i_and_m_g_status': 'PFS'}, inplace=True)
ib_out.rename(columns={'os_g_status': 'OS', 'pfs_i_and_m_g_status': 'PFS'}, inplace=True)
folfox_out.rename(columns={'os_g_status': 'OS', 'pfs_i_and_m_g_status': 'PFS'}, inplace=True)
#add prefix to egfr_out columns except PFS and OS
egfr_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in egfr_out.columns]
vegf_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in vegf_out.columns]
ib_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in ib_out.columns]
folfox_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in folfox_out.columns]
egfr_out.head()

,id_record_id,id_institution,id_drugs_list,OS,PFS,id_sample_id
0,GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,1.0,GENIE-DFCI-000971-10958
1,GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,1.0,GENIE-DFCI-001292-7406
2,GENIE-DFCI-001463,DFCI,Cetuximab,1,0.0,GENIE-DFCI-001463-9650
3,GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,1.0,GENIE-DFCI-001463-9650
4,GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",1,1.0,GENIE-DFCI-002507-4815


In [30]:
mut = pd.read_csv('../data/crc_mutoh_pertreat.csv', index_col=0)
mut.columns = ['mut_' + i for i in mut.columns]
cna = pd.read_csv('../data/crc_cna_pertreat.csv', index_col=0)
cna.columns = ['cna_' + i for i in cna.columns]
clin = pd.read_csv('../data/crc_clin_pub.csv', index_col=0)
#add prefix to clin columns except PFS and OS
clin.columns = ['clin_' + i if i not in ['PFS', 'OS', 'OS_time'] else i for i in clin.columns]
#drop 'OS' and 'PFS' columns
clin = clin.drop(['OS', 'PFS'], axis=1)
cat_cols = [col for col in clin.columns if col.startswith('clin') and clin[col].nunique() <= 10]
num_cols = [col for col in clin.columns if col.startswith('clin') and clin[col].nunique() > 10]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in cat_cols:
    clin[i] = le.fit_transform(clin[i])
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
for i in num_cols:
    clin[i] = imp.fit_transform(clin[i].values.reshape(-1,1))

In [31]:
mut.head()

,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,mut_MLH1,mut_PNRC1,mut_ETV1,mut_TSC1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_KMT2D,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1
GENIE-DFCI-002643-6598,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
GENIE-DFCI-008624-7126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GENIE-DFCI-003423-6929,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GENIE-DFCI-007675-6942,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
GENIE-DFCI-008889-6970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
cna.head()

,cna_ABL1,cna_AKT1,cna_AKT2,cna_AKT3,cna_ALK,cna_ALOX12B,cna_APC,cna_AR,cna_ARAF,cna_ARID1A,...,cna_TMPRSS2,cna_TNFAIP3,cna_TP53,cna_TSC1,cna_TSC2,cna_TSHR,cna_U2AF1,cna_VHL,cna_WT1,cna_XPO1
sample_id,,,,,,,,,,,,,,,,,,,,,
GENIE-DFCI-002643-6598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GENIE-DFCI-008624-7126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GENIE-DFCI-003423-6929,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,...,-1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
GENIE-DFCI-007675-6942,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
GENIE-DFCI-008889-6970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
egfr = egfr_out.set_index('id_sample_id')
vegf = vegf_out.set_index('id_sample_id')
ib = ib_out.set_index('id_sample_id')
folfox = folfox_out.set_index('id_sample_id')

In [34]:
egfr_mut = egfr.join(mut, how='inner')
egfr_mut_cna = egfr_mut.join(cna, how='inner')
print(egfr_mut_cna.shape)

(354, 453)


In [35]:
vegf_mut = vegf.join(mut, how='inner')
vegf_mut_cna = vegf_mut.join(cna, how='inner')
print(vegf_mut_cna.shape)

ib_mut = ib.join(mut, how='inner')
ib_mut_cna = ib_mut.join(cna, how='inner')
print(ib_mut_cna.shape)

folfox_mut = folfox.join(mut, how='inner')
folfox_mut_cna = folfox_mut.join(cna, how='inner')
print(folfox_mut_cna.shape)

(922, 453)
(146, 453)
(1383, 453)


In [36]:
egfr_mut_cna_clin = egfr_mut_cna.set_index('id_record_id').join(clin, how='inner')
vegf_mut_cna_clin = vegf_mut_cna.set_index('id_record_id').join(clin, how='inner')
ib_mut_cna_clin = ib_mut_cna.set_index('id_record_id').join(clin, how='inner')
folfox_mut_cna_clin = folfox_mut_cna.set_index('id_record_id').join(clin, how='inner')
print(egfr_mut_cna_clin.shape, vegf_mut_cna_clin.shape, ib_mut_cna_clin.shape, folfox_mut_cna_clin.shape)

(354, 474) (922, 474) (146, 474) (1383, 474)


In [37]:
egfr_mut_cna_clin.head()

,id_institution,id_drugs_list,OS,PFS,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,1.0,0,0,0,0,0,0,...,0,1,0,0,1,814,0,0,4,9.3
GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,1.0,0,0,0,0,0,0,...,5,2,3,0,2,637,0,0,4,20.0
GENIE-DFCI-001463,DFCI,Cetuximab,1,0.0,0,0,0,0,0,0,...,5,2,2,0,0,5468,0,0,1,0.5
GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,1.0,0,0,0,0,0,0,...,5,2,2,0,0,5468,0,0,1,0.5
GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",1,1.0,0,0,0,0,0,0,...,0,2,2,0,0,2288,0,0,3,83.4


In [38]:
vegf_mut_cna_clin.head()

,id_institution,id_drugs_list,OS,PFS,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000147,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",1,1.0,0,0,0,0,0,0,...,5,0,2,0,2,1235,0,0,1,2.8
GENIE-DFCI-000233,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",1,0.0,0,0,0,0,0,0,...,5,2,2,0,2,3303,0,0,1,1.0
GENIE-DFCI-000306,DFCI,"Bevacizumab, Fluorouracil, Leucovorin Calcium,...",1,1.0,0,0,0,0,0,0,...,1,2,2,0,3,1230,0,0,1,0.7
GENIE-DFCI-000738,DFCI,"Bevacizumab, Fluorouracil, Irinotecan Hydrochl...",0,0.0,0,0,0,0,0,0,...,5,0,2,0,3,1623,0,1,1,1.8
GENIE-DFCI-000946,DFCI,"Bevacizumab, Fluorouracil",1,0.0,0,0,0,0,0,0,...,5,2,3,1,0,817,0,0,1,5.5


In [39]:
ib_mut_cna_clin.head()

,id_institution,id_drugs_list,OS,PFS,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000971,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,0,1,0,0,1,814,0,0,4,9.3
GENIE-DFCI-001038,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,0,2,0,0,0,1465,0,0,4,100.0
GENIE-DFCI-002499,DFCI,Cabozantinib Smalate,1,0.0,0,0,0,0,0,0,...,1,1,1,1,0,1343,0,0,1,100.0
GENIE-DFCI-002507,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,0,2,2,0,0,2288,0,0,3,83.4
GENIE-DFCI-002561,DFCI,Regorafenib,1,1.0,0,0,0,0,0,0,...,0,1,2,1,0,2140,0,0,1,8.4


In [40]:
folfox_mut_cna_clin.head()

,id_institution,id_drugs_list,OS,PFS,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000147,DFCI,"Capecitabine, Oxaliplatin",1,NaN,0,0,0,0,0,0,...,5,0,2,0,2,1235,0,0,1,2.8
GENIE-DFCI-000233,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,1.0,0,0,0,0,0,0,...,5,2,2,0,2,3303,0,0,1,1.0
GENIE-DFCI-000247,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,0.0,0,0,0,0,0,0,...,0,2,0,0,2,1163,0,0,0,4.9
GENIE-DFCI-000306,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,1.0,0,0,0,0,0,0,...,1,2,2,0,3,1230,0,0,1,0.7
GENIE-DFCI-000738,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,NaN,0,0,0,0,0,0,...,5,0,2,0,3,1623,0,1,1,1.8


In [41]:
egfr_mut_cna_clin.to_csv('../data/crc_egfr_mut_cna_clin.csv')
vegf_mut_cna_clin.to_csv('../data/crc_vegf_mut_cna_clin.csv')
ib_mut_cna_clin.to_csv('../data/crc_ib_mut_cna_clin.csv')
folfox_mut_cna_clin.to_csv('../data/crc_folfox_mut_cna_clin.csv')